In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import string
from textblob import TextBlob
from gensim import matutils, models
import scipy.sparse

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)
sns.set()

In [16]:
rel_path = '..\\dataset\\'

dir_list = os.listdir(rel_path)
print(type(dir_list))
print(len(dir_list))
print((dir_list[0]))

<class 'list'>
546
df_review_AB Aviation.csv


In [17]:
df = pd.DataFrame()

for file in dir_list:
    df_temp = pd.read_csv(f'{rel_path}{file}', sep=';')
    
    ## --- concat
    df = pd.concat([df, df_temp], axis=0, join='outer', ignore_index=True)

In [19]:
df.columns

Index(['airline', 'id', 'review', 'rating', 'header', 'sub_header', 'author',
       'time_published', 'Type Of Traveller', 'Seat Type', 'Route',
       'Date Flown', 'Seat Comfort', 'Cabin Staff Service', 'Food & Beverages',
       'Ground Service', 'Value For Money', 'Recommended', 'Aircraft',
       'Inflight Entertainment', 'Wifi & Connectivity'],
      dtype='object')

In [20]:
## --- preprocess

def search_city(x):
    try:
        x = re.compile(r'\(.*\)').search(x).group()
        x = x.replace('(','').replace(')','')
    except Exception as e:
        x = ''
    return x


def type_aircraft_search(x):
    if type(x) == type('a'):
        if ('A' in x) and ('Boeing' in x):
            x = 'A / Boeing'
        elif ('A' in x) or ('Boeing' in x):
            if ('A' in x):
                x = 'A'
            else:
                x = 'Boeing'
        elif 'Embraer' in x:
            x = 'Embraer'
    return x

df['verified'] = df['review'].apply(lambda x: x.split(' |')[0])
df['verified'] = df['verified'].str.replace('✅ ','')
df['rating_only'] = df['rating'].apply(lambda x: x.split('/')[0] if type(x)==type('a') else x)
df['rating_only'] = df['rating_only'].astype('float')
df['header'] = df['header'].str.replace('"','')
df['city'] = df['sub_header'].apply(lambda x: search_city(x))
df['type_aircraft'] = df['Aircraft'].apply(lambda x: type_aircraft_search(x))

# df.drop(labels='review', axis=1, inplace=True)
df.columns = [col_name.lower().replace(' ','_') for col_name in df.columns]

In [23]:
# create functions for cleaning purposes

def cleaning_text_round_1(text):
    '''lowercase, remove punctuation, special character, words with number, '''
    
    text = text.lower()
    text = re.compile(r'[%s]' % string.punctuation).sub('', text)
    text = text.replace('✅ ','')
    text = re.compile(r'\w*\d+\w*').sub('', text)
    return text


def cleaning_text_round_2(text):
    '''additional punctuation and non-sensical text'''
    
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

In [28]:
df['review_clean'] = np.vectorize(cleaning_text_round_1)(df['review'])
df['review_clean'] = np.vectorize(cleaning_text_round_2)(df['review_clean'])

In [32]:
# sentiment & polarity

pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

df['polarity'] = df['review_clean'].apply(pol)
df['subjectivity'] = df['review_clean'].apply(sub)

In [36]:
def clean_date(date):
    split_date = date.split(' ')
    split_date[0] = split_date[0].replace('st', '').replace('nd', '').replace('rd', '').replace('th', '')
    date = ' '.join(split_date)
    return date
    
    
df['time_published'] = df['time_published'].apply(clean_date)
df['time_published'] = pd.to_datetime(df['time_published'], format='%d %B %Y')

In [37]:
df.head()

,airline,id,review,rating,header,sub_header,author,time_published,type_of_traveller,seat_type,route,date_flown,seat_comfort,cabin_staff_service,food_&_beverages,ground_service,value_for_money,recommended,aircraft,inflight_entertainment,wifi_&_connectivity,verified,rating_only,city,type_aircraft,review_clean,polarity,subjectivity
0,AB Aviation,anchor694481,✅ Trip Verified | Moroni to Moheli. Turned ou...,9/10,pretty decent airline,Tom Hansen (Netherlands) 11th November 2019,Tom Hansen,2019-11-11,Solo Leisure,Economy Class,Moroni to Moheli,November 2019,4.0000,5.0000,4.0000,4.0000,3.0000,yes,NaN,NaN,NaN,Trip Verified,9.0000,,NaN,trip verified moroni to moheli turned out to...,0.3022,0.6261
1,AB Aviation,anchor666859,✅ Trip Verified | Moroni to Anjouan. It is a v...,1/10,Not a good airline,26 reviews Gyan Fernando (United Kingdom) 25th...,26 reviewsGyan Fernando,2019-06-25,Solo Leisure,Economy Class,Moroni to Anjouan,June 2019,2.0000,2.0000,1.0000,1.0000,2.0000,no,E120,NaN,NaN,Trip Verified,1.0000,,E120,trip verified moroni to anjouan it is a very ...,-0.0771,0.4183
2,AB Aviation,anchor666802,✅ Trip Verified | Anjouan to Dzaoudzi. A very...,1/10,flight was fortunately short,26 reviews Gyan Fernando (United Kingdom) 25th...,26 reviewsGyan Fernando,2019-06-25,Solo Leisure,Economy Class,Anjouan to Dzaoudzi,June 2019,2.0000,1.0000,1.0000,1.0000,2.0000,no,Embraer E120,NaN,NaN,Trip Verified,1.0000,,Embraer,trip verified anjouan to dzaoudzi a very sma...,0.0458,0.5367
3,Adria Airways,anchor685541,Not Verified | Please do a favor yourself and...,1/10,I will never fly again with Adria,D Praetextatus (Serbia) 28th September 2019,D Praetextatus,2019-09-28,Solo Leisure,Economy Class,Frankfurt to Pristina,September 2019,1.0000,1.0000,NaN,1.0000,1.0000,no,NaN,NaN,NaN,Not Verified,1.0000,,NaN,not verified please do a favor yourself and ...,-0.1937,0.5823
4,Adria Airways,anchor684542,✅ Trip Verified | Do not book a flight with th...,1/10,it ruined our last days of holidays,D Meijer (Netherlands) 24th September 2019,D Meijer,2019-09-24,Couple Leisure,Economy Class,Sofia to Amsterdam via Ljubljana,September 2019,1.0000,1.0000,1.0000,1.0000,1.0000,no,NaN,1.0000,1.0000,Trip Verified,1.0000,,NaN,trip verified do not book a flight with this ...,0.1750,0.5229


In [38]:
df.describe(include='number')

,seat_comfort,cabin_staff_service,food_&_beverages,ground_service,value_for_money,inflight_entertainment,wifi_&_connectivity,rating_only,polarity,subjectivity
count,111335.0000,111255.0000,90427.0000,83895.0000,123037.0000,73183.0000,27787.0000,121085.0000,125422.0000,125422.0000
mean,2.7936,3.0408,2.7804,2.4505,2.7060,2.6903,2.0058,4.6021,0.0894,0.5127
std,1.4462,1.5856,1.4922,1.6003,1.6027,1.5073,1.4320,3.4995,0.2134,0.1284
min,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,-1.0000,0.0000
25%,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,-0.0392,0.4346
50%,3.0000,3.0000,3.0000,2.0000,3.0000,3.0000,1.0000,3.0000,0.0803,0.5093
75%,4.0000,5.0000,4.0000,4.0000,4.0000,4.0000,3.0000,8.0000,0.2250,0.5879
max,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,10.0000,1.0000,1.0000


In [39]:
df.describe(exclude='number')

C:\Users\ichfa\AppData\Local\Temp\ipykernel_18732\1805651458.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(exclude='number')


,airline,id,review,rating,header,sub_header,author,time_published,type_of_traveller,seat_type,route,date_flown,recommended,aircraft,verified,city,type_aircraft,review_clean
count,125422,125422,125422,121085,125422,125422,125422,125422,87104,122401,86799,86951,125422,35242,125422,125422,35242,125422
unique,546,125343,125140,11,67831,119325,89996,5076,4,4,48318,107,2,3474,53784,1,709,125132
top,American Airlines,anchor317177,Aug 28 from Baltimore to Boston. Check-in staf...,1/10,Spirit Airlines customer review,29 reviews A Berlov (Russian Federation) 19th ...,Anders Pedersen,2015-01-19 00:00:00,Solo Leisure,Economy Class,Sydney to Melbourne,August 2015,no,A320,Trip Verified,,A,rating cabin flown economy value for money s...
freq,5403,2,2,41883,1066,26,251,297,31847,99298,144,1556,74526,4368,42579,125422,16203,7
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# drop unnecessary columns

df.drop(labels = ['id', 'review', 'rating', 'header', 'sub_header', 'author', 
                  'route', 'date_flown', 'aircraft', 'verified',
                 'city', 'type_aircraft'],
       axis=1,
       inplace=True)

In [41]:
df.head()

,airline,time_published,type_of_traveller,seat_type,seat_comfort,cabin_staff_service,food_&_beverages,ground_service,value_for_money,recommended,inflight_entertainment,wifi_&_connectivity,rating_only,review_clean,polarity,subjectivity
0,AB Aviation,2019-11-11,Solo Leisure,Economy Class,4.0000,5.0000,4.0000,4.0000,3.0000,yes,NaN,NaN,9.0000,trip verified moroni to moheli turned out to...,0.3022,0.6261
1,AB Aviation,2019-06-25,Solo Leisure,Economy Class,2.0000,2.0000,1.0000,1.0000,2.0000,no,NaN,NaN,1.0000,trip verified moroni to anjouan it is a very ...,-0.0771,0.4183
2,AB Aviation,2019-06-25,Solo Leisure,Economy Class,2.0000,1.0000,1.0000,1.0000,2.0000,no,NaN,NaN,1.0000,trip verified anjouan to dzaoudzi a very sma...,0.0458,0.5367
3,Adria Airways,2019-09-28,Solo Leisure,Economy Class,1.0000,1.0000,NaN,1.0000,1.0000,no,NaN,NaN,1.0000,not verified please do a favor yourself and ...,-0.1937,0.5823
4,Adria Airways,2019-09-24,Couple Leisure,Economy Class,1.0000,1.0000,1.0000,1.0000,1.0000,no,1.0000,1.0000,1.0000,trip verified do not book a flight with this ...,0.1750,0.5229


In [42]:
# save file for dashboard purposes

df.to_csv(f'{rel_path}cleaned_df.csv', index=False)